In [158]:
#Cargamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [159]:
df = pd.read_csv('Mexico_Limpio.csv')

In [160]:
#Leemos las bases de datos
limpio = pd.read_csv('Mexico_Limpio.csv')
original = pd.read_csv('Original_Mexico.csv')

VARIABLES ELEGIDAS
Mexico_Limpio = 'host_is_superhost', 'accommodates', 'bathrooms', 'bedrooms', 'beds','room_type' 
Original_Mexico = 'host_response_time', 'host_verifications', 'instant_bookable', 'host_since'  

Limpia de datos y conversción a tipo dicotómicas

In [161]:
elegidas = limpio[['host_is_superhost', 'accommodates', 'bathrooms', 'bedrooms', 'beds','room_type']]
elegidas.head()

,host_is_superhost,accommodates,bathrooms,bedrooms,beds,room_type
0,0.0,2,1.00,1.0,1.0,Entire home/apt
1,0.0,7,2.25,3.5,3.5,Entire home/apt
2,0.0,2,1.00,1.0,1.0,Entire home/apt
3,0.0,4,1.00,2.0,2.0,Entire home/apt
4,1.0,2,1.00,1.0,1.0,Entire home/apt


In [162]:
print('Lista de valores únicos de las columnas de interés')

#La columna 'host_is_superhost' nos indica la capacidad de huéspedes
print('host_is_superhost:',limpio['host_is_superhost'].unique())

#La columna 'accommodates' nos indica la capacidad de huéspedes
print('accommodates:',limpio['accommodates'].unique())

#La columna 'bathrooms' nos indica el número de baños
print('bathrooms:', limpio['bathrooms'].unique())

#La columna 'bedrooms' nos indica el número de habitaciones
print('bedrooms:', limpio['bedrooms'].unique())

#La columna 'beds' nos indica el número de camas
print('beds:', limpio['beds'].unique())

#La columna 'room_type' nos indica el tipo de alojamiento 
print('room_type:', limpio['room_type'].unique())

Lista de valores únicos de las columnas de interés
host_is_superhost: [0.        1.        0.3864308]
accommodates: [2 7 4 3 1 5 6]
bathrooms: [1.         2.25       1.45038927 1.5        2.         0.5
 0.25      ]
bedrooms: [1.        3.5       2.        0.        1.5550889 3.       ]
beds: [1.         3.5        2.         1.95321156 3.         0.        ]
room_type: ['Entire home/apt' 'Private room' 'Hotel room' 'Shared room']


In [163]:
#Tratamos columna 'host_is_superhost'
#Pasamos la columna a cadena de texto
limpio['host_is_superhost'] = limpio['host_is_superhost'].astype(str)

#Cambiamos los que tienen el valor de '0.386430796' a valores invalidos para susittuirlos
limpio['host_is_superhost'] = limpio['host_is_superhost'].replace({'0.386430796': np.nan})

#Rellena los NaN usando el valor anterior (Forward Fill)
limpio['host_is_superhost'] = limpio['host_is_superhost'].fillna(method='ffill')
#Rellena los NaN usando el valor posterior (Backward Fill)
limpio['host_is_superhost'] = limpio['host_is_superhost'].fillna(method='bfill')

limpio['host_is_superhost'] = limpio['host_is_superhost'].replace({'0.0': 'Es Superhost', '1.0':'No es Superhost'}) 
#Verificamos valores unicos
print('-------host_is_superhost:',limpio['host_is_superhost'].unique())


#Tratamos columna 'accommodates'
#Pasamos la columna a cadena de texto
limpio['accommodates'] = limpio['accommodates'].astype(str)
limpio['accommodates'] = limpio['accommodates'].replace({'1': 'Alojamiento para Parejas/Individual', '2':'Alojamiento para Parejas/Individual',
                                                         '3': 'Alojamiento para Familias/Grupos', '4': 'Alojamiento para Familias/Grupos', '5': 'Alojamiento para Familias/Grupos', 
                                                         '6': 'Alojamiento para Familias/Grupos', '7': 'Alojamiento para Familias/Grupos' }) 
#Verificamos valores únicos
print('-------accommodates:',limpio['accommodates'].unique())

#Tratamos columna 'bathrooms'
#Pasamos la columna a cadena de texto
limpio['bathrooms'] = limpio['bathrooms'].astype(str)
limpio['bathrooms'] = limpio['bathrooms'].replace({'0.5': 'Menos de un baño completo', '0.25':'Menos de un baño completo',
                                                    '1.0': 'Uno o más baños completos', '2.0': 'Uno o más baños completos', '1.5': 'Uno o más baños completos', 
                                                    '1.450389273': 'Uno o más baños completos', '2.25': 'Uno o más baños completos' }) 
#Verificamos valores únicos
print('-------bathrooms:',limpio['bathrooms'].unique())


#Tratamos columna 'bedrooms'
#Pasamos la columna a cadena de texto
limpio['bedrooms'] = limpio['bedrooms'].astype(str)
limpio['bedrooms'] = limpio['bedrooms'].replace({'0.0': 'Dos o menos de dos cuartos', '2.0':'Dos o menos de dos cuartos',
                                                    '1.0': 'Dos o menos de dos cuartos', '3.0': 'Más de dos cuartos', '3.5': 'Más de dos cuartos', 
                                                    '1.555088904': 'Dos o menos de dos cuartos'}) 
#Verificamos valores únicos
print('-------bedrooms:',limpio['bedrooms'].unique())

#Tratamos columna 'beds'
#Pasamos la columna a cadena de texto
limpio['beds'] = limpio['beds'].astype(str)
limpio['beds'] = limpio['beds'].replace({'0.0': 'Dos o menos de dos camas', '2.0':'Dos o menos de dos camas',
                                                    '1.0': 'Dos o menos de dos camas', '3.0': 'Más de dos camas', '3.5': 'Más de dos camas', 
                                                    '1.953211565': 'Dos o menos de dos camas'}) 
#Verificamos valores únicos
print('-------beds:',limpio['beds'].unique())

#Tratamos columna 'room_type'
#Pasamos la columna a cadena de texto
limpio['room_type'] = limpio['room_type'].astype(str)
limpio['room_type'] = limpio['room_type'].replace({'Entire home/apt': 'Alojamiento Completo', 'Hotel room': 'Alojamiento Completo',
                                                   'Private room': 'Alojamiento Parcial', 'Shared room': 'Alojamiento Parcial'}) 
#Verificamos valores únicos
print('-------room_type :',limpio['room_type'].unique())

-------host_is_superhost: ['Es Superhost' 'No es Superhost']
-------accommodates: ['Alojamiento para Parejas/Individual' 'Alojamiento para Familias/Grupos']
-------bathrooms: ['Uno o más baños completos' 'Menos de un baño completo']
-------bedrooms: ['Dos o menos de dos cuartos' 'Más de dos cuartos']
-------beds: ['Dos o menos de dos camas' 'Más de dos camas']
-------room_type : ['Alojamiento Completo' 'Alojamiento Parcial']


C:\Users\vbece\AppData\Local\Temp\ipykernel_10428\3467536860.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  limpio['host_is_superhost'] = limpio['host_is_superhost'].fillna(method='ffill')
C:\Users\vbece\AppData\Local\Temp\ipykernel_10428\3467536860.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  limpio['host_is_superhost'] = limpio['host_is_superhost'].fillna(method='bfill')


In [164]:
#Agarramos la variables objetivo que necesitamos de la base original, las tratamos y agregamos a la base limpia
cambio = original[['host_response_time', 'host_verifications', 'instant_bookable', 'host_since']]
cambio.head()

,host_response_time,host_verifications,instant_bookable,host_since
0,NaN,"['email', 'phone', 'work_email']",f,28/06/2010
1,within an hour,"['email', 'phone', 'work_email']",f,09/08/2010
2,within a few hours,"['email', 'phone']",f,19/10/2010
3,within a few hours,"['email', 'phone']",f,04/01/2011
4,within a few hours,"['email', 'phone', 'work_email']",f,24/08/2010


In [165]:
#La columna 'host_since' nos habla de que fecha es dado de alta en la plataforma y es anfitrión 
cambio['host_since'].isnull().sum()

np.int64(914)

In [166]:
#Rellena los NaN usando el valor anterior (Forward Fill)
cambio['host_since'] = cambio['host_since'].fillna(method='ffill')

#Rellena los NaN usando el valor posterior (Backward Fill)
cambio['host_since'] = cambio['host_since'].fillna(method='bfill')

#Verifcamos trata de NAN's
print('Número de valores nulos:', cambio['host_since'].isnull().sum())
#Transformamos a tipo date
cambio['host_since'] = pd.to_datetime(cambio['host_since'])

#Conseguimos fecha 'promedio'
fecha_mas_repetida = cambio['host_since'].mode().iloc[0]
print("La fecha de alta de anfitrión que más se repite es:", fecha_mas_repetida)

Número de valores nulos: 0
La fecha de alta de anfitrión que más se repite es: 2023-01-30 00:00:00


C:\Users\vbece\AppData\Local\Temp\ipykernel_10428\2929194693.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cambio['host_since'] = cambio['host_since'].fillna(method='ffill')
C:\Users\vbece\AppData\Local\Temp\ipykernel_10428\2929194693.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cambio['host_since'] = cambio['host_since'].fillna(method='ffill')
C:\Users\vbece\AppData\Local\Temp\ipykernel_10428\2929194693.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cambio['host_since'] = cambio['host_since'].fillna(method='bfill')
C:\Users\vbece\AppData\Local\Temp\ipyk

In [167]:
#Definimos fecha de partido
FECHA_CORTE = pd.to_datetime('2023-01-30')

# Convertimos la columna de texto 'host_since' a datetime, especificando el formato
host_since_dt = pd.to_datetime(cambio['host_since'], format='%d/%m/%Y', errors='coerce')

# #Hacemos el ajuste de las fechas
cambio['host_since_agrupada'] = np.where(host_since_dt < FECHA_CORTE, 'Antes del 30/01/2023','Después del 30/01/2023')

# #Verificamos cambios
print('Los valores unicos de la columna agrupada: ', cambio['host_since_agrupada'].unique())
#Verificamos valores nulos después de todas las modificaciones
cambio['host_since'].isnull().sum()

Los valores unicos de la columna agrupada:  ['Antes del 30/01/2023' 'Después del 30/01/2023']


C:\Users\vbece\AppData\Local\Temp\ipykernel_10428\260167155.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cambio['host_since_agrupada'] = np.where(host_since_dt < FECHA_CORTE, 'Antes del 30/01/2023','Después del 30/01/2023')


np.int64(0)

In [168]:
#La columna de 'host_verifications' nos indica los métodos de verificación que tiene el anfitrión
print('Lista de valores únicos de la columna:',cambio['host_verifications'].unique())

Lista de valores únicos de la columna: ["['email', 'phone', 'work_email']" "['email', 'phone']" "['phone']" nan
 "['phone', 'work_email']" "['email']" '[]' "['email', 'work_email']"
 "['work_email']"]


In [169]:
#Tratamos columna 'host_verifications'
cambio['host_verifications'] = cambio['host_verifications'].replace({"['email', 'phone', 'work_email']": 'Más de un método', "['email', 'phone']": 'Más de un método', "['phone', 'work_email']": 'Más de un método', "['email', 'work_email']": 'Más de un método',
                                                                    "['phone']": 'Un método', "['email']": 'Un método', "['work_email']": 'Un método' }) 
#Verificamos cambio y existencia de valores nulos
print('Lista de valores únicos:',cambio['host_verifications'].unique())
cambio['host_verifications'].isnull().sum()

Lista de valores únicos: ['Más de un método' 'Un método' nan '[]']


C:\Users\vbece\AppData\Local\Temp\ipykernel_10428\3062924664.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cambio['host_verifications'] = cambio['host_verifications'].replace({"['email', 'phone', 'work_email']": 'Más de un método', "['email', 'phone']": 'Más de un método', "['phone', 'work_email']": 'Más de un método', "['email', 'work_email']": 'Más de un método',


np.int64(914)

In [170]:
#Cambiamos los que tienen [] como si fueran valores invalidos para susittuirlos
cambio['host_verifications'] = cambio['host_verifications'].replace({'[]': np.nan})

#Rellena los NaN usando el valor anterior (Forward Fill)
cambio['host_verifications'] = cambio['host_verifications'].fillna(method='ffill')

#Rellena los NaN usando el valor posterior (Backward Fill)
cambio['host_verifications'] = cambio['host_verifications'].fillna(method='bfill')

#Verificamos que ya no existan valores nulos
cambio['host_verifications'].isnull().sum()

C:\Users\vbece\AppData\Local\Temp\ipykernel_10428\2851848341.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cambio['host_verifications'] = cambio['host_verifications'].replace({'[]': np.nan})
C:\Users\vbece\AppData\Local\Temp\ipykernel_10428\2851848341.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cambio['host_verifications'] = cambio['host_verifications'].fillna(method='ffill')
C:\Users\vbece\AppData\Local\Temp\ipykernel_10428\2851848341.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

np.int64(0)

In [171]:
#La columna 'instant_bookable' hace referencia a si el huésped puede reservar automáticamente el anuncio sin la intervención del anfitrión
print('Lista de valores únicos de la columna:',cambio['instant_bookable'].unique())

Lista de valores únicos de la columna: ['f' 't']


In [172]:
#Tratamos columna 'instant_bookable'
cambio['instant_bookable'] = cambio['instant_bookable'].replace({'t': 'Reserva Automática', 'f': 'No Reserva Automática'})

#Verificamos cambio y existencia de valores nulos
print('Lista de valores únicos:',cambio['instant_bookable'].unique())
cambio['instant_bookable'].isnull().sum()

Lista de valores únicos: ['No Reserva Automática' 'Reserva Automática']


C:\Users\vbece\AppData\Local\Temp\ipykernel_10428\654513735.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cambio['instant_bookable'] = cambio['instant_bookable'].replace({'t': 'Reserva Automática', 'f': 'No Reserva Automática'})


np.int64(0)

In [173]:
#La columna 'host_response_time' se refiere a cuanto tarda el anfitrión en contestar
print('Lista de valores únicos de la columna:',cambio['host_response_time'].unique())

Lista de valores únicos de la columna: [nan 'within an hour' 'within a few hours' 'a few days or more'
 'within a day']


In [174]:
#Tratamos columna 'host_response_time'
cambio['host_response_time'] = cambio['host_response_time'].replace({'within an hour': 'Horas de Respuesta','within a few hours': 'Horas de Respuesta', 
                                                                     'a few days or more': 'Días de Respuesta', 'within a day': 'Días de Respuesta'})

#Verificamos cambios
print('Lista de valores únicos:',cambio['host_response_time'].unique())
#Verificamos valores nulos
cambio['host_response_time'].isnull().sum()

Lista de valores únicos: [nan 'Horas de Respuesta' 'Días de Respuesta']


C:\Users\vbece\AppData\Local\Temp\ipykernel_10428\3597048255.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cambio['host_response_time'] = cambio['host_response_time'].replace({'within an hour': 'Horas de Respuesta','within a few hours': 'Horas de Respuesta',


np.int64(4686)

In [175]:
#Rellena los NaN usando el valor anterior (Forward Fill)
cambio['host_response_time'] = cambio['host_response_time'].fillna(method='ffill')
#Rellena los NaN usando el valor posterior (Backward Fill)
cambio['host_response_time'] = cambio['host_response_time'].fillna(method='bfill')

#Verificamos de nuevo los valores nulos 
print('Número de valores nulos:',cambio['host_response_time'].isnull().sum())
#Verificamos cambios
print('Lista de valores únicos:',cambio['host_response_time'].unique())

Número de valores nulos: 0
Lista de valores únicos: ['Horas de Respuesta' 'Días de Respuesta']


C:\Users\vbece\AppData\Local\Temp\ipykernel_10428\1120293309.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cambio['host_response_time'] = cambio['host_response_time'].fillna(method='ffill')
C:\Users\vbece\AppData\Local\Temp\ipykernel_10428\1120293309.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cambio['host_response_time'] = cambio['host_response_time'].fillna(method='ffill')
C:\Users\vbece\AppData\Local\Temp\ipykernel_10428\1120293309.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cambio['host_response_time'] = cambio['host_response_time'].fillna(method

Correlación Lógistica 

VARIABLES ELEGIDAS
Mexico_Limpio = 'host_is_superhost', 'accommodates', 'bathrooms', 'bedrooms', 'beds','room_type' 
Original_Mexico = 'host_response_time', 'host_verifications', 'instant_bookable', 'host_since' 

In [176]:
limpio.columns

Index(['id', 'scrape_id', 'host_id', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_listings_count',
       'host_total_listings_count', 'latitude', 'longitude', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'price', 'minimum_nights',
       'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'availability_30',
       'availability_60', 'availability_90', 'availability_365',
       'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
       'availability_eoy', 'number_of_reviews_ly', 'estimated_occupancy_l365d',
       'estimated_revenue_l365d', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value',
       'calculated_host_listings_count',
    

In [177]:
#Hacemos copia de la base de datos original seleciconada a la base limpia
limpio['host_response_time'] = cambio['host_response_time']
limpio['host_verifications'] = cambio['host_verifications']
limpio['instant_bookable'] = cambio['instant_bookable']
limpio['host_since'] = cambio['host_since']

In [191]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= limpio[['estimated_occupancy_l365d', 'number_of_reviews_ltm']]
Var_Dep= limpio['host_since']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None) 

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión de la variables host_since:')
print(matriz)

#Calculo la precisión del modelo
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average="micro", pos_label="Antes del 30/01/2023")
print('Precisión del modelo:', precision)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:', exactitud)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="micro", pos_label="Después del 30/01/2023")
print('Sensibilidad del modelo:', sensibilidad)

Matriz de Confusión de la variables host_since:
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Precisión del modelo: 0.01918949627572276
Exactitud del modelo: 0.01918949627572276


c:\Users\vbece\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1784: UserWarning: Note that pos_label (set to 'Antes del 30/01/2023') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
c:\Users\vbece\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1784: UserWarning: Note that pos_label (set to 'Después del 30/01/2023') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


Sensibilidad del modelo: 0.01918949627572276


In [178]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= limpio[['number_of_reviews', 'review_scores_communication']]
Var_Dep= limpio['host_response_time']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None) 

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión de la variables host_response_time:')
print(matriz)

#Calculo la precisión del modelo
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average="binary", pos_label="Horas de Respuesta")
print('Precisión del modelo:', precision)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:', exactitud)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Días de Respuesta")
print('Sensibilidad del modelo:', sensibilidad)

print('---------------Aplicando Reponderación de las clases------------------')

#Reponderación de las clases
algoritmo_Pond = LogisticRegression(class_weight='balanced')
algoritmo_Pond.fit(X_train, y_train)
#Realizamos una predicción
y_pred_pond = algoritmo_Pond.predict(X_test) 

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz_2 = confusion_matrix(y_test, y_pred_pond)
print('Matriz de Confusión con Reponderación de clases:')
print(matriz_2)
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred_pond, average="binary", pos_label="Horas de Respuesta")
print('Precisión del modelo:')
print(precision) 

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred_pond)
print('Exactitud del modelo:')
print(exactitud)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred_pond, average="binary", pos_label="Días de Respuesta")
print('Sensibilidad del modelo:')
print(sensibilidad)

Matriz de Confusión de la variables host_response_time:
[[   0  702]
 [   0 7219]]
Precisión del modelo: 0.9113748264108067
Exactitud del modelo: 0.9113748264108067
Sensibilidad del modelo: 0.0
---------------Aplicando Reponderación de las clases------------------
Matriz de Confusión con Reponderación de clases:
[[ 580  122]
 [3982 3237]]
Precisión del modelo:
0.9636796665674308
Exactitud del modelo:
0.4818836005554854
Sensibilidad del modelo:
0.8262108262108262


In [179]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= limpio[['review_scores_communication', 'number_of_reviews']]
Var_Dep= limpio['host_verifications']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None) 

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión de la variables host_verifications:')
print(matriz)

#Calculo la precisión del modelo
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average="binary", pos_label="Más de un método")
print('Precisión del modelo:', precision)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:', exactitud)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Un método")
print('Sensibilidad del modelo:', sensibilidad)


print('---------------Aplicando Reponderación de las clases------------------')

#Reponderación de las clases
algoritmo_Pond = LogisticRegression(class_weight='balanced')
algoritmo_Pond.fit(X_train, y_train)
#Realizamos una predicción
y_pred_pond = algoritmo_Pond.predict(X_test) 

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz_2 = confusion_matrix(y_test, y_pred_pond)
print('Matriz de Confusión con Reponderación de clases:')
print(matriz_2)
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred_pond, average="binary", pos_label="Más de un método")
print('Precisión del modelo:')
print(precision) 

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred_pond)
print('Exactitud del modelo:')
print(exactitud)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred_pond, average="binary", pos_label="Un método")
print('Sensibilidad del modelo:')
print(sensibilidad)

Matriz de Confusión de la variables host_verifications:
[[7174    0]
 [ 747    0]]
Precisión del modelo: 0.9056937255397046
Exactitud del modelo: 0.9056937255397046
Sensibilidad del modelo: 0.0
---------------Aplicando Reponderación de las clases------------------
Matriz de Confusión con Reponderación de clases:
[[3072 4102]
 [ 207  540]]
Precisión del modelo:
0.9368709972552608
Exactitud del modelo:
0.4560030299204646
Sensibilidad del modelo:
0.7228915662650602


In [180]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= limpio[['reviews_per_month', 'review_scores_value']]
Var_Dep= limpio['instant_bookable']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None) 

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión de la variables instant_bookable:')
print(matriz)

#Calculo la precisión del modelo
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average="binary", pos_label="Reserva Automática")
print('Precisión del modelo:', precision)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:', exactitud)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="No Reserva Automática")
print('Sensibilidad del modelo:', sensibilidad)

Matriz de Confusión de la variables instant_bookable:
[[4059  556]
 [2467  839]]
Precisión del modelo: 0.6014336917562724
Exactitud del modelo: 0.6183562681479611
Sensibilidad del modelo: 0.8795232936078007


In [181]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= limpio[['estimated_revenue_l365d', 'number_of_reviews']]
Var_Dep= limpio['room_type']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None) 

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión de la variables room_type:')
print(matriz)

#Calculo la precisión del modelo
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average="binary", pos_label="Alojamiento Completo")
print('Precisión del modelo:', precision)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:', exactitud)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Alojamiento Parcial")
print('Sensibilidad del modelo:', sensibilidad)

Matriz de Confusión de la variables room_type:
[[4047 1159]
 [1435 1280]]
Precisión del modelo: 0.7382342210871945
Exactitud del modelo: 0.6725160964524681
Sensibilidad del modelo: 0.4714548802946593


In [182]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= limpio[['estimated_revenue_l365d', 'price']]
Var_Dep= limpio['beds']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None) 

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión de la variables beds:')
print(matriz)

#Calculo la precisión del modelo
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average="binary", pos_label="Dos o menos de dos camas")
print('Precisión del modelo:', precision)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:', exactitud)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Más de dos camas")
print('Sensibilidad del modelo:', sensibilidad)

Matriz de Confusión de la variables beds:
[[6313  105]
 [1259  244]]
Precisión del modelo: 0.833729529846804
Exactitud del modelo: 0.8277995202625931
Sensibilidad del modelo: 0.16234198270126413


In [183]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= limpio[['price', 'maximum_nights']]
Var_Dep= limpio['bedrooms']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None) 

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión de la variables bedrooms:')
print(matriz)

#Calculo la precisión del modelo
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average="binary", pos_label="Dos o menos de dos cuartos")
print('Precisión del modelo:', precision)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:', exactitud)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Más de dos cuartos")
print('Sensibilidad del modelo:', sensibilidad)

Matriz de Confusión de la variables bedrooms:
[[6976  166]
 [ 573  206]]
Precisión del modelo: 0.9240959067426149
Exactitud del modelo: 0.9067036990279005
Sensibilidad del modelo: 0.2644415917843389


In [184]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= limpio[['price', 'availability_365']]
Var_Dep= limpio['bathrooms']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None) 

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión de la variables bathrooms:')
print(matriz)

#Calculo la precisión del modelo
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average="binary", pos_label="Uno o más baños completos")
print('Precisión del modelo:', precision)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:', exactitud)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label='Menos de un baño completo')
print('Sensibilidad del modelo:', sensibilidad)

print('---------------Aplicando Reponderación de las clases------------------')

#Reponderación de las clases
algoritmo_Pond = LogisticRegression(class_weight='balanced')
algoritmo_Pond.fit(X_train, y_train)
#Realizamos una predicción
y_pred_pond = algoritmo_Pond.predict(X_test) 

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz_2 = confusion_matrix(y_test, y_pred_pond)
print('Matriz de Confusión con Reponderación de clases:')
print(matriz_2)
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred_pond, average="binary", pos_label="Uno o más baños completos")
print('Precisión del modelo:')
print(precision) 

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred_pond)
print('Exactitud del modelo:')
print(exactitud)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred_pond, average="binary", pos_label="Menos de un baño completo")
print('Sensibilidad del modelo:')
print(sensibilidad)


Matriz de Confusión de la variables bathrooms:
[[   0   63]
 [   0 7858]]
Precisión del modelo: 0.992046458780457
Exactitud del modelo: 0.992046458780457
Sensibilidad del modelo: 0.0
---------------Aplicando Reponderación de las clases------------------
Matriz de Confusión con Reponderación de clases:
[[  44   19]
 [4075 3783]]
Precisión del modelo:
0.9950026301946344
Exactitud del modelo:
0.48314606741573035
Sensibilidad del modelo:
0.6984126984126984


In [185]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= limpio[['price', 'availability_30']]
Var_Dep= limpio['accommodates']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None) 

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión de la variables accomodates:')
print(matriz)

#Calculo la precisión del modelo
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average="binary", pos_label="Alojamiento para Parejas/Individual")
print('Precisión del modelo:', precision)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:', exactitud)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Alojamiento para Familias/Grupos")
print('Sensibilidad del modelo:', sensibilidad)

Matriz de Confusión de la variables accomodates:
[[2092 1726]
 [1142 2961]]
Precisión del modelo: 0.6317473863878814
Exactitud del modelo: 0.6379245044817573
Sensibilidad del modelo: 0.5479308538501834


In [186]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= limpio[['host_response_rate', 'reviews_per_month' ]]
Var_Dep= limpio['host_is_superhost']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None) 

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión de la variables host_is_superhost:')
print(matriz)

#Calculo la precisión del modelo
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average="binary", pos_label="Es Superhost")
print('Precisión del modelo:', precision)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:', exactitud)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="No es Superhost")
print('Sensibilidad del modelo:', sensibilidad)

Matriz de Confusión de la variables host_is_superhost:
[[4113  763]
 [1888 1157]]
Precisión del modelo: 0.6853857690384936
Exactitud del modelo: 0.6653200353490721
Sensibilidad del modelo: 0.37996715927750413
